In [1]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Deploy ADK Agent in AI Engine

This notebook provides a step-by-step to deploy an Agent Created using Agent Development Kit on Agent Engine (ReasonEngine on Vertex AI)

**Important**: This notebook consider that the Agent was built with ADK and the agent files are inside an agent folder and the dependencies in a file config.yaml

Folders structure (example): 
```
parent_folder/
    agent_folder/
        __init__.py
        agent.py
        config.yaml
        .env
    deploy_agent_engine.ipynb
```

config.yaml (example): 
```yaml
agent_name: 'agent_name'
agent_display_name: 'Agent Name'
agent_description: 'Useful agent to help users'

deploy:
  dependencies: ['google-cloud-aiplatform[agent_engines]', 'google-adk', 'cloudpickle', 'pydantic']
```


### Setup and Config

In [2]:
# Checking the google-adk and google-cloud-aiplatform versions
!pip freeze | grep google-adk
!pip freeze | grep google-cloud-aiplatform

google-adk==1.3.0
google-cloud-aiplatform==1.97.0


In [3]:
# Authentication on gcloud
!gcloud auth application-default login

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=g2oD1ns76LndEeLg4Ahk6UNgJC4JeY&access_type=offline&code_challenge=mRkzg8ZzaBoXkz6CFGYtwoWa_Nqehc5c_Uqu25ux1Y0&code_challenge_method=S256


Credentials saved to file: [/Users/speca/.config/gcloud/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).

Quota project "speca-sandbox-argolis" was added to ADC which can be used by Google client libraries for billing and quota. Note that some services may still bill the project owning the resource.


In [4]:
# Basic Libraries
import os 
import vertexai
import yaml

# AI Engine on Vertex AI 
from vertexai import agent_engines

# Library for AI Engine with ADK
from vertexai.preview import reasoning_engines

# To view JSON response formatted
import json
from IPython.display import display,Markdown,JSON


In [5]:
# Load Agent Config
AGENT_DIR = "goes_fiscal_agent"

# Load environment variables from .env file from agent Directory 
from dotenv import load_dotenv
env_file = f'./{AGENT_DIR}/.env'
load_dotenv(env_file)

# Load config from agents (Params and dependencies for deploy)
with open(f'./{AGENT_DIR}/config.yaml', 'r') as f:
    config = yaml.safe_load(f)

# For Vertex AI SDK 
PROJECT_ID = "speca-sandbox-argolis"
LOCATION = "us-central1"
BUCKET = "gs://speca-argolis-adk-agents"

### Instantiate Agent from Directory

In [6]:
# Importing Agent Module from AGENT_DIR folder
import importlib
agent_module = importlib.import_module(f"{AGENT_DIR}.agent")

# Instantiate the Assistant as an ADK App 
adk_agent = reasoning_engines.AdkApp(
    agent=agent_module.root_agent,
)

### Running Agent Local (Optional)

In [7]:
# Run a simple query
for event in adk_agent.stream_query(
    user_id="user_1",
    message="Hola o que puedes hacer?",
):
    pass

# Formatted output
display(Markdown(f"```json\n{json.dumps(event, indent=2,ensure_ascii=False)}\n```"))

```json
{
  "content": {
    "parts": [
      {
        "text": "Hola, soy un agente especializado en el análisis de datos fiscales del Gobierno de El Salvador. Puedo ayudarte a obtener información sobre la ejecución del presupuesto público, gastos por institución, área de gestión, fuente de financiamiento y diferentes etapas presupuestarias como lo programado, modificado, comprometido y devengado.\n\n¿Tienes alguna pregunta específica sobre el presupuesto o el gasto público?"
      }
    ],
    "role": "model"
  },
  "usage_metadata": {
    "candidates_token_count": 76,
    "candidates_tokens_details": [
      {
        "modality": "TEXT",
        "token_count": 76
      }
    ],
    "prompt_token_count": 1491,
    "prompt_tokens_details": [
      {
        "modality": "TEXT",
        "token_count": 1491
      }
    ],
    "thoughts_token_count": 79,
    "total_token_count": 1646,
    "traffic_type": "ON_DEMAND"
  },
  "invocation_id": "e-e95a9966-e816-47da-bc27-dc76e97378cd",
  "author": "goes_fiscal_agent",
  "actions": {
    "state_delta": {},
    "artifact_delta": {},
    "requested_auth_configs": {}
  },
  "id": "7iaU5dU5",
  "timestamp": 1750044479.507381
}
```

### Deploy on Agent Engine

In [8]:
# Instantiate Vertex AI
vertexai.init(
    project=PROJECT_ID,
    location=LOCATION,
    staging_bucket=BUCKET,
)

In [9]:
## Retrieve all existent Agent Engine on your project
for agent in agent_engines.list():
    print(f"============================ \nAgent: {agent.display_name}\nResoruce Name: {agent.resource_name}\nCreated/updated at: {agent.update_time} \n\n" )

Agent: Procurement Analytics Agent
Resoruce Name: projects/809256159830/locations/us-central1/reasoningEngines/2791853536969752576
Created/updated at: 2025-06-05 17:22:44.496217+00:00 


Agent: Legal Analytics Agent
Resoruce Name: projects/809256159830/locations/us-central1/reasoningEngines/5506961162320740352
Created/updated at: 2025-05-31 15:40:31.286244+00:00 


Agent: Data Assistant
Resoruce Name: projects/809256159830/locations/us-central1/reasoningEngines/7923142362405011456
Created/updated at: 2025-05-31 12:08:18.375253+00:00 




In [10]:
# Read Requirements for Agent from config file
# Usually this ['google-cloud-aiplatform[agent_engines]', 'google-adk', 'cloudpickle'] plus the packages that agent needs
requirements = config['deploy']['dependencies']
requirements

['google-cloud-aiplatform[agent_engines,adk]',
 'google-adk',
 'cloudpickle',
 'pydantic',
 'google-cloud-bigquery',
 'pandas',
 'db-dtypes',
 'pyyaml']

In [11]:
# Extra packages from agent folder (This is all .py files inside Agent Directory)
extra_packages = [AGENT_DIR]
extra_packages

['goes_fiscal_agent']

In [12]:
# Load Variables on env_vars dict to be used when creating the Agent
env_vars = {}

env_vars["BQ_PROJECT_ID"] = os.getenv("BQ_PROJECT_ID")
env_vars["BQ_DATASET_ID"] = os.getenv("BQ_DATASET_ID")
env_vars["AGENT_ROOT_MODEL"] = os.getenv("AGENT_ROOT_MODEL")
env_vars["AGENT_TOOL_MODEL"] = os.getenv("AGENT_TOOL_MODEL")

env_vars

{'BQ_PROJECT_ID': 'speca-sandbox-argolis',
 'BQ_DATASET_ID': 'goes_data',
 'AGENT_ROOT_MODEL': 'gemini-2.5-flash-preview-04-17',
 'AGENT_TOOL_MODEL': 'gemini-2.5-flash-preview-04-17'}

In [ ]:
## (OPTIONAL) - Update the agent at the same Agent Resource 
# agent_engines.update(
#     resource_name=remote_agent.resource_name,    # Required.
#     agent_engine = adk_agent,             # The Agent instantiated as ADK agent
#     requirements=requirements,            # Requirements file
#     extra_packages=extra_packages,        # Extra packages
#     display_name=config['agent_name'],    # Display name  
#     description=config['agent_name'],     # Description
#     env_vars=env_vars                     # Env Vars dict
# )

In [13]:
# Deploy the Agent on AI Engine (This takes a few minutes)
remote_agent = agent_engines.create(
    agent_engine = adk_agent,             # The Agent instantiated as ADK agent
    requirements=requirements,            # Requirements file
    extra_packages=extra_packages,        # Extra packages
    display_name=config['agent_display_name'],    # Display name  
    description=config['agent_description'],     # Description
    env_vars=env_vars                     # Env Vars dict
)

Identified the following requirements: {'cloudpickle': '3.1.1', 'google-cloud-aiplatform': '1.97.0', 'pydantic': '2.11.7'}
The final list of requirements: ['google-cloud-aiplatform[agent_engines,adk]', 'google-adk', 'cloudpickle', 'pydantic', 'google-cloud-bigquery', 'pandas', 'db-dtypes', 'pyyaml']
Using bucket speca-argolis-adk-agents
Wrote to gs://speca-argolis-adk-agents/agent_engine/agent_engine.pkl
Writing to gs://speca-argolis-adk-agents/agent_engine/requirements.txt
Creating in-memory tarfile of extra_packages
Writing to gs://speca-argolis-adk-agents/agent_engine/dependencies.tar.gz
Creating AgentEngine
Create AgentEngine backing LRO: projects/809256159830/locations/us-central1/reasoningEngines/3333341023416877056/operations/4175457906843451392
View progress and logs at https://console.cloud.google.com/logs/query?project=speca-sandbox-argolis
AgentEngine created. Resource name: projects/809256159830/locations/us-central1/reasoningEngines/3333341023416877056
To use this AgentEng

### Test Remote Agent on Agent Engine

In [14]:
## Retrieve all existent Agent Engine resource.names (Agents)
for agent in agent_engines.list():
    print(f"============================ \nAgent: {agent.display_name}\nResoruce Name: {agent.resource_name}\nCreated/updated at: {agent.update_time} \n\n" )

Agent: Fiscal Agent Gobierno de El Salvador
Resoruce Name: projects/809256159830/locations/us-central1/reasoningEngines/3333341023416877056
Created/updated at: 2025-06-16 03:34:31.816880+00:00 


Agent: Procurement Analytics Agent
Resoruce Name: projects/809256159830/locations/us-central1/reasoningEngines/2791853536969752576
Created/updated at: 2025-06-05 17:22:44.496217+00:00 


Agent: Legal Analytics Agent
Resoruce Name: projects/809256159830/locations/us-central1/reasoningEngines/5506961162320740352
Created/updated at: 2025-05-31 15:40:31.286244+00:00 


Agent: Data Assistant
Resoruce Name: projects/809256159830/locations/us-central1/reasoningEngines/7923142362405011456
Created/updated at: 2025-05-31 12:08:18.375253+00:00 




In [ ]:
#remote_agent = agent_engines.get("projects/809256159830/locations/us-central1/reasoningEngines/2791853536969752576")

In [15]:
print(f"=================== Remote Agent ============================ \n\
 Name: {remote_agent.display_name}\n\
 Resoruce Name: {remote_agent.resource_name}\n\
 Created/updated at: {remote_agent.update_time} \n\n" )

=================== Remote Agent ============================ 
 Name: Fiscal Agent Gobierno de El Salvador
 Resoruce Name: projects/809256159830/locations/us-central1/reasoningEngines/3333341023416877056
 Created/updated at: 2025-06-16 03:34:31.816880+00:00 




In [16]:
# Run a simple query
for remote_event in remote_agent.stream_query(
    user_id="speca@google.com",
    message="Hola, o que puedes hacer?",
):
    display(JSON(remote_event,expanded=True)) 

<IPython.core.display.JSON object>

In [17]:
# Formatted output
display(Markdown(f"```json\n{json.dumps(remote_event, indent=2,ensure_ascii=False)}\n```"))

```json
{
  "content": {
    "parts": [
      {
        "text": "Hola. Soy un agente especializado en el análisis de datos de ejecución presupuestaria del Gobierno de El Salvador.\n\nPuedo ayudarte a consultar información sobre:\n*   **Instituciones:** Datos de gasto por institución gubernamental.\n*   **Áreas de Gestión:** Información sobre el presupuesto y gasto por áreas funcionales del gobierno.\n*   **Etapas Presupuestarias:** Montos en las diferentes etapas como Programado, Modificado, Compromiso y Devengado.\n*   **Fuentes de Financiamiento:** Desglose del gasto según el origen de los fondos (Fondo General, Préstamos, Donaciones, etc.).\n*   **Unidades Presupuestarias y Líneas de Trabajo:** Detalles a niveles más específicos de la estructura presupuestaria.\n*   **Tiempo:** Análisis por año y mes.\n\nPuedes preguntarme, por ejemplo:\n*   ¿Cuál fue el monto devengado por el Ministerio de Salud en 2023?\n*   ¿Cuánto se comprometió en el área de Desarrollo Social el año pasado?\n*   ¿Cuál es el presupuesto modificado para el Ministerio de Educación este año?\n*   ¿Cómo se distribuye el gasto devengado por fuente de financiamiento en un mes específico?\n\nEstoy listo para ayudarte con tus consultas sobre la ejecución del presupuesto."
      }
    ],
    "role": "model"
  },
  "usage_metadata": {
    "candidates_token_count": 273,
    "candidates_tokens_details": [
      {
        "modality": "TEXT",
        "token_count": 273
      }
    ],
    "prompt_token_count": 1492,
    "prompt_tokens_details": [
      {
        "modality": "TEXT",
        "token_count": 1492
      }
    ],
    "thoughts_token_count": 246,
    "total_token_count": 2011,
    "traffic_type": "ON_DEMAND"
  },
  "invocation_id": "e-8c30414b-f640-4110-bfb0-f63df5580d4e",
  "author": "goes_fiscal_agent",
  "actions": {
    "state_delta": {},
    "artifact_delta": {},
    "requested_auth_configs": {}
  },
  "id": "6tR7xm8R",
  "timestamp": 1750044957.355378
}
```